# 0. 초기 설정
- 드라이브 마운트
- 모듈 임포트
- gpu 연결(학습시)
- 데이터 폴더 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader 
from torch.utils.data import random_split
from torchvision import datasets 
from torchvision import transforms 
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torchvision.models as models
from copy import deepcopy
import numpy as np
import pandas as pd

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# 1. 데이터 전처리
- 변환기(transform)
- train, valid, test 나누기
- 

# 2. 시각화
- 이미지&라벨링 확인

# 3. 배치
- dataload(batch_size, shuffle)

# 4. model 설정
- 모델 작성 또는 전이학습

# 5. 학습

# 6. 결과예측

# 7. test

# 8. model 저장